<a href="https://colab.research.google.com/github/wtaisner/tensorflow-great-barrier-reef/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import wandb
except:
    !pip install wandb
    import wandb
!wandb login

wandb: Currently logged in as: annprz (use `wandb login --relogin` to force relogin)


In [2]:
try:
    import torchmetrics
except:
    !pip install torchmetrics
    import torchmetrics

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
from copy import deepcopy
import os
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision
import ast
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from torchmetrics.detection.map import MeanAveragePrecision
from torchvision.transforms import ToPILImage


# this should probably be changed to something smart, right?
KAGGLE_PATH_ANNOTATIONS = '/kaggle/input/tensorflow-great-barrier-reef/train.csv'
KAGGLE_PATH_IMG_DIR = '/kaggle/input/tensorflow-great-barrier-reef/train_images/'
LOCAL_PATH_ANNOTATIONS = 'data/train.csv'
LOCAL_PATH_IMG_DIR = 'data/train_images/'
COLAB_PATH_ANNOTATIONS = '/content/drive/MyDrive/data/train.csv'
COLAB_PATH_IMG_DIR = '/content/drive/MyDrive/data/train_images/'

In [5]:
class StarfishDataset(Dataset):
    def __init__(self,
                 annotations_file=COLAB_PATH_ANNOTATIONS,
                 img_dir=COLAB_PATH_IMG_DIR
                 ):
        self.img_labels = pd.read_csv(annotations_file)
        self.annotated = self.img_labels[self.img_labels['annotations'] != '[]']  # get only annotated frames
        self.img_dir = img_dir

    def __len__(self):
        return len(self.annotated)

    def __getitem__(self, idx):
        image = read_image(os.path.join(self.img_dir, 'video_{}'.format(self.annotated.iloc[idx][0]),
                                        '{}.jpg'.format(self.annotated.iloc[idx][2])))
        min_image = image.min()
        max_image = image.max()
        # normalize image to 0-1 - required by torchvision
        image -= min_image
        image = image/max_image
        labels = self.annotated.iloc[idx][-1]
        labels = ast.literal_eval(labels)
        coords = []
        for parsed_label in labels:
            x, y = parsed_label['x'], parsed_label['y']
            w, h = parsed_label['width'], parsed_label['height']
            cx, cy = 0.5*w + x, 0.5*h + y
            coords.append([0, cx/1280, cy/720, w/1280, h/720])

        boxes = np.array(coords)
        return image, boxes


In [6]:
torch.manual_seed(23)

# IF YOU WANT TO RUN PROPER MODEL LEARNING, MAKE SURE TO CHANGE DATASET SIZES

dataset = StarfishDataset()
print(len(dataset))
train_size = 1000
test_size = (len(dataset) - train_size)//2
val_size = len(dataset) - train_size - test_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_size, val_size, test_size))

# extract only small part of the data for faster learning / testing process
# train_size = int(0.8 * len(train_dataset))
# test_size = len(train_dataset) - train_size
# train_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, test_size])


print('Train dataset: {} instances, validation dataset: {}, test dataset: {}'.format(len(train_dataset), len(val_dataset), len(test_dataset)))

4919
Train dataset: 1000 instances, validation dataset: 1960, test dataset: 1959


In [7]:
def prepare_dataset(dataset, path_img, path_lbl):
  i = 0
  for (image, label) in dataset:
    file_image = path_img + '/im' + str(i) + '.jpg'
    file_label = path_lbl + '/im' + str(i) + '.txt'
    image = ToPILImage()(image)
    image.save(file_image)
    np.savetxt(file_label, label, fmt='%i %.4f %.4f %.4f %.4f')
    i += 1

In [8]:
%cd
%cd /content/drive/MyDrive/data_yolo

/root
/content/drive/MyDrive/data_yolo


In [9]:
train_path_img = 'images/train'
train_path_lbl = 'labels/train'

prepare_dataset(train_dataset, train_path_img, train_path_lbl)

In [ ]:
val_path_img = 'images/val'
val_path_lbl = 'labels/val'

prepare_dataset(val_dataset, val_path_img, val_path_lbl)

In [ ]:
test_path_img = 'images/test'
test_path_lbl = 'labels/test'

prepare_dataset(test_dataset, test_path_img, test_path_lbl)

In [ ]:
%cd
%cd /content/drive/MyDrive
%pwd

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

In [ ]:
!python train.py --img 1280 --batch 32 --epochs 10 --data ../data_yolo.yaml --weights yolov5s.pt --cache --project "great-barrier-reef" --name "randomrandom" --entity "ap-wt"